In [1]:
""" Python main script of MatMat trade module

    Notes
    ------
    Fill notes if necessary

    """

' Python main script of MatMat trade module\n\n    Notes\n    ------\n    Fill notes if necessary\n\n    '

# Imports

In [2]:
# general
import sys
import os
import copy

# scientific
import numpy as np
import pandas as pd
import pymrio
import matplotlib.pyplot as plt

# local folder
from local_paths import data_dir
from local_paths import output_dir

# local library
from utils import Tools

# SETTINGS

In [3]:
# year to study in [*range(1995, 2022 + 1)]
base_year = 2015

# system type: pxp or ixi
system = 'pxp'

# agg name: to implement in agg_matrix.xlsx
agg_name = {
	'sector': 'ref',
	'region': 'ref'
}

# define filename concatenating settings
concat_settings = str(base_year) + '_' + \
	agg_name['sector']  + '_' +  \
	agg_name['region']

# set if rebuilding calibration from exiobase
calib = False

# READ/ORGANIZE/CLEAN DATA

In [4]:
# define file name
file_name = 'IOT_' + str(base_year) + '_' + system + '.zip'


# download data online
if not os.path.isfile(data_dir / file_name):

	pymrio.download_exiobase3(
	    storage_folder = data_dir,
	    system = system, 
	    years = base_year
	)


# import or build calibration data
if calib:

	# import exiobase data
	reference = pymrio.parse_exiobase3(
		data_dir / file_name
	)

	# isolate ghg emissions
	reference.ghg_emissions = Tools.extract_ghg_emissions(reference)

	# del useless extensions
	reference.remove_extension(['satellite', 'impacts'])

	# import agregation matrices
	agg_matrix = {
		key: pd.read_excel(
			data_dir / 'agg_matrix.xlsx',
			sheet_name = key + '_' + value
		) for (key, value) in agg_name.items()
	}
	agg_matrix['sector'].set_index(['category', 'sub_category', 'sector'], inplace = True)
	agg_matrix['region'].set_index(['Country name', 'Country code'], inplace = True)

	# apply regional and sectorial agregations
	reference.aggregate(
		region_agg = agg_matrix['region'].T.values,
		sector_agg = agg_matrix['sector'].T.values,
		region_names = agg_matrix['region'].columns.tolist(),
		sector_names = agg_matrix['sector'].columns.tolist()
	)

	# reset all to flows before saving
	reference = reference.reset_to_flows()
	reference.ghg_emissions.reset_to_flows()

	# save calibration data
	reference.save_all(
		data_dir / ('reference' + '_' + concat_settings)
	)

else:

	# import calibration data built with calib = True
	reference = pymrio.parse_exiobase3(
		data_dir / ('reference' + '_' + concat_settings)
	)


# CALCULATIONS

In [5]:
# calculate reference system
reference.calc_all()


# update extension calculations
reference.ghg_emissions_desag = Tools.recal_extensions_per_region(
	reference,
	'ghg_emissions'
)

# init counterfactual(s)
counterfactual = reference.copy()
counterfactual.remove_extension('ghg_emissions_desag')


sectors_list = list(reference.get_sectors())
reg_list = list(reference.get_regions())

# read param sets to shock reference system
## ToDo


# build conterfactual(s) using param sets
## ToDo


# calculate counterfactual(s) system
counterfactual.calc_all()
counterfactual.ghg_emissions_desag = Tools.recal_extensions_per_region(
	counterfactual,
	'ghg_emissions'
)

# VISUALIZE

In [6]:
dcba = reference.ghg_emissions_desag.D_cba
dpba = reference.ghg_emissions_desag.D_pba
dimp = reference.ghg_emissions_desag.D_imp
dexp = reference.ghg_emissions_desag.D_exp

# Desagrégation spatiale de l'enfer

In [7]:
reag_matrix = pd.read_excel(data_dir / 'agg_matrix_opti.xlsx', sheet_name = 'region_ref')
reag_matrix.columns[2:]

Index(['FR', 'United Kingdom', 'United States', 'Asia and Row Europe',
       'Chinafrica', 'Turkey and RoW America', 'Pacific and RoW Middle East',
       'Brazil, Mexico and South Africa', 'Switzerland and Norway',
       'RoW Asia and Pacific', 'EU'],
      dtype='object')

In [8]:
#create dic for region reaggregation :
dict_reag={}
dict_reag['FR']=['FR']
for reg_agg in list(reag_matrix.columns[3:]):
    #print(reg_agg)
    list_reg_agg = []
    for i in reag_matrix.index:
        reg = reag_matrix.iloc[i].loc['Country name']
        #print(reg)
        if reag_matrix[reg_agg].iloc[i] == 1:
            list_reg_agg.append(reg)
    #print(list_reg_agg)
    dict_reag[reg_agg]=list_reg_agg
dict_reag

{'FR': ['FR'],
 'United Kingdom': ['United Kingdom'],
 'United States': ['United States'],
 'Asia and Row Europe': ['Japan',
  'India',
  'Russia',
  'Indonesia',
  'RoW Europe'],
 'Chinafrica': ['China', 'RoW Africa'],
 'Turkey and RoW America': ['Canada', 'Turkey', 'RoW America'],
 'Pacific and RoW Middle East': ['South Korea',
  'Australia',
  'Taiwan',
  'RoW Middle East'],
 'Brazil, Mexico and South Africa': ['Brazil', 'Mexico', 'South Africa'],
 'Switzerland and Norway': ['Switzerland', 'Norway'],
 'RoW Asia and Pacific': ['RoW Asia and Pacific'],
 'EU': ['Austria',
  'Belgium',
  'Bulgaria',
  'Cyprus',
  'Czech Republic',
  'Germany',
  'Denmark',
  'Estonia',
  'Spain',
  'Finland',
  'Greece',
  'Croatia',
  'Hungary',
  'Ireland',
  'Italy',
  'Lithuania',
  'Luxembourg',
  'Latvia',
  'Malta',
  'Netherlands',
  'Poland',
  'Portugal',
  'Romania',
  'Sweden',
  'Slovenia',
  'Slovakia']}

In [9]:
#create dic for region reaggregation, reversed i.e. for each region, dic gives its new group :
dict_reag_new_group={}

for reg_agg in dict_reag :
    for reg in dict_reag[reg_agg]:
        dict_reag_new_group[reg] = reg_agg
#for sec in sectors_list:
#    dict_reag_new_group[sec] = sec
dict_reag_new_group

{'FR': 'FR',
 'United Kingdom': 'United Kingdom',
 'United States': 'United States',
 'Japan': 'Asia and Row Europe',
 'India': 'Asia and Row Europe',
 'Russia': 'Asia and Row Europe',
 'Indonesia': 'Asia and Row Europe',
 'RoW Europe': 'Asia and Row Europe',
 'China': 'Chinafrica',
 'RoW Africa': 'Chinafrica',
 'Canada': 'Turkey and RoW America',
 'Turkey': 'Turkey and RoW America',
 'RoW America': 'Turkey and RoW America',
 'South Korea': 'Pacific and RoW Middle East',
 'Australia': 'Pacific and RoW Middle East',
 'Taiwan': 'Pacific and RoW Middle East',
 'RoW Middle East': 'Pacific and RoW Middle East',
 'Brazil': 'Brazil, Mexico and South Africa',
 'Mexico': 'Brazil, Mexico and South Africa',
 'South Africa': 'Brazil, Mexico and South Africa',
 'Switzerland': 'Switzerland and Norway',
 'Norway': 'Switzerland and Norway',
 'RoW Asia and Pacific': 'RoW Asia and Pacific',
 'Austria': 'EU',
 'Belgium': 'EU',
 'Bulgaria': 'EU',
 'Cyprus': 'EU',
 'Czech Republic': 'EU',
 'Germany': 'EU',

In [10]:
M = reference.ghg_emissions_desag.M.copy()
ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']


#create pd.MultiIndex for new_M
multi_reg = []
multi_sec = []
for reg in list(reag_matrix.columns[2:]) :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))
new_M = pd.DataFrame(np.zeros((len(ghg_list),len(sectors_list)*len(list(reag_matrix.columns[2:])))),
                              index =reference.ghg_emissions_desag.M.index,columns = new_col)


new_M['FR']=M['FR']
M
#print(M.columns)
    
for reg_agg in dict_reag:
    #print('\n', reg_agg)
    list_reg_agg = dict_reag[reg_agg]
    #print(list_reg_agg)
    for reg2 in list_reg_agg :
        #print(reg2)
        new_M[reg_agg] += M[reg2]/len(list_reg_agg)

new_M

region            FR                                                           \
sector   Agriculture Crude coal     Crude oil Natural gas Extractive industry   
stressor                                                                        
CO2         0.000590   0.003463  5.621522e-04    0.000903            0.000443   
CH4         0.001496   0.000118  4.279977e-03    0.000152            0.000066   
N2O         0.000875   0.000031  1.193598e-05    0.000011            0.000015   
SF6         0.000004   0.000002  1.310750e-06    0.000002            0.000026   
HFC         0.000134   0.000044  2.262622e-05    0.000014            0.001298   
PFC         0.000002   0.000001  8.968207e-07    0.000002            0.000009   

region                                                                      \
sector   Biomass_industry  Clothing Heavy_industry Construction Automobile   
stressor                                                                     
CO2              0.000429  0.000353       0.000729     0.000286   0.000377   
CH4              0.000373  0.000092       0.000110     0.000078   0.000080   
N2O              0.000196  0.000023       0.000019     0.000017   0.000014   
SF6              0.000002  0.000001       0.000003     0.000002   0.000003   
HFC              0.000056  0.000015       0.000070     0.000030   0.000018   
PFC              0.000002  0.000002       0.000004     0.000003   0.000003   

region    ...             EU                                                  \
sector    ... Heavy_industry Construction Automobile Oth transport equipment   
stressor  ...                                                                  
CO2       ...       0.000916     0.000346   0.000375                0.000552   
CH4       ...       0.000145     0.000075   0.000078                0.000068   
N2O       ...       0.000016     0.000010   0.000010                0.000010   
SF6       ...       0.000012     0.000004   0.000004                0.000003   
HFC       ...       0.000042     0.000019   0.000014                0.000012   
PFC       ...       0.000031     0.000007   0.000005                0.000004   

region                                                            \
sector   Machinery Electronics Fossil fuels Electricity and heat   
stressor                                                           
CO2       0.000358    0.000360     0.002934             0.003129   
CH4       0.000068    0.000061     0.001157             0.000263   
N2O       0.000009    0.000009     0.000012             0.000033   
SF6       0.000003    0.000002     0.000008             0.000006   
HFC       0.000012    0.000009     0.000030             0.000018   
PFC       0.000004    0.000003     0.000007             0.000003   

region                                 
sector   Transport services Composite  
stressor                               
CO2                0.000819  0.000207  
CH4                0.000086  0.000080  
N2O                0.000009  0.000008  
SF6                0.000002  0.000002  
HFC                0.000008  0.000011  
PFC                0.000002  0.000002  

[6 rows x 187 columns]

In [11]:
S = reference.ghg_emissions_desag.S.copy()
ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']


#create pd.MultiIndex for new_S
multi_reg = []
multi_sec = []
for reg in list(reag_matrix.columns[2:]) :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))
new_S = pd.DataFrame(np.zeros((len(ghg_list),len(sectors_list)*len(list(reag_matrix.columns[2:])))),
                              index =reference.ghg_emissions_desag.S.index,columns = new_col)


new_S['FR']=S['FR']
S
#print(S.columns)
    
for reg_agg in dict_reag:
    #print('\n', reg_agg)
    list_reg_agg = dict_reag[reg_agg]
    #print(list_reg_agg)
    for reg2 in list_reg_agg :
        #print(reg2)
        new_S[reg_agg] += S[reg2]/len(list_reg_agg)

new_S

region              FR                                            \
sector     Agriculture    Crude coal     Crude oil   Natural gas   
stressor                                                           
CO2       2.752560e-04  3.210168e-03  3.354614e-04  5.992512e-04   
CH4       1.165031e-03  6.867609e-07  4.196768e-03  5.487065e-05   
N2O       7.196057e-04  2.072283e-05  5.189820e-06  2.943933e-07   
SF6       1.937657e-06  6.007967e-07  2.313296e-07  2.450581e-08   
HFC       1.034492e-04  3.217736e-05  1.401206e-05  1.257714e-06   
PFC       5.991865e-07  2.060316e-07  1.191699e-07  9.961699e-09   

region                                                                      \
sector   Extractive industry Biomass_industry      Clothing Heavy_industry   
stressor                                                                     
CO2             1.973631e-04     9.696668e-05  3.881976e-05   3.386476e-04   
CH4             5.066902e-07     3.193208e-06  5.591624e-07   1.760052e-06   
N2O             1.245002e-06     4.259090e-06  4.191614e-08   1.261228e-06   
SF6             2.420474e-05     3.464996e-07  3.176694e-08   2.747924e-07   
HFC             1.263246e-03     1.821899e-05  1.678283e-06   2.671380e-05   
PFC             7.261031e-06     1.074834e-07  9.150018e-09   1.117455e-06   

region                                ...             EU                \
sector    Construction    Automobile  ... Heavy_industry  Construction   
stressor                              ...                                
CO2       1.488762e-05  1.828081e-05  ...   3.886385e-04  5.911019e-05   
CH4       7.804559e-08  1.473360e-07  ...   9.974999e-06  5.584754e-06   
N2O       3.866893e-08  1.061538e-07  ...   9.873284e-07  4.449358e-07   
SF6       0.000000e+00  0.000000e+00  ...   4.467649e-06  2.430965e-07   
HFC       0.000000e+00  0.000000e+00  ...   1.268184e-05  1.148011e-06   
PFC       0.000000e+00  0.000000e+00  ...   1.632818e-05  4.029455e-07   

region                                                                      \
sector      Automobile Oth transport equipment     Machinery   Electronics   
stressor                                                                     
CO2       4.413741e-05            2.417216e-04  7.187185e-05  9.823409e-05   
CH4       1.603817e-06            1.160142e-06  2.949714e-06  1.952853e-06   
N2O       7.517405e-07            8.012262e-07  5.416436e-07  9.543006e-07   
SF6       5.925973e-08            6.786831e-08  1.881098e-07  5.722308e-08   
HFC       5.219439e-07            3.367309e-07  1.217698e-06  7.556944e-07   
PFC       1.946813e-08            3.908375e-08  3.964075e-07  4.465302e-07   

region                                                                        
sector    Fossil fuels Electricity and heat Transport services     Composite  
stressor                                                                      
CO2       2.406675e-03         2.437386e-03       5.675150e-04  4.462630e-05  
CH4       5.528467e-04         2.018583e-05       2.566451e-06  3.416558e-05  
N2O       9.469407e-07         2.214930e-05       2.980392e-06  5.510486e-07  
SF6       2.910415e-06         1.225997e-07       6.577529e-08  1.005559e-06  
HFC       1.464568e-05         8.296012e-07       4.950624e-07  4.540700e-06  
PFC       4.281670e-06         9.426002e-08       1.905644e-07  8.226175e-07  

[6 rows x 187 columns]

In [12]:
list_reg_reag_new=list(reag_matrix.columns[2:])
list_reg_reag_new

['FR',
 'United Kingdom',
 'United States',
 'Asia and Row Europe',
 'Chinafrica',
 'Turkey and RoW America',
 'Pacific and RoW Middle East',
 'Brazil, Mexico and South Africa',
 'Switzerland and Norway',
 'RoW Asia and Pacific',
 'EU']

In [13]:
dcba = reference.ghg_emissions_desag.D_cba.copy()

ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']

multi_reg = []
multi_sec = []
for reg in list_reg_reag_new :
    for sec in sectors_list :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_reag_new :
    for ghg in ghg_list :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

new_dcba = pd.DataFrame(np.zeros((len(ghg_list)*len(list_reg_reag_new),
                                  len(sectors_list)*len(list_reg_reag_new))),
                              index =new_index,columns = new_col)

for reg_export in dict_reag :
    list_reg_agg_1 = dict_reag[reg_export]
    for reg_import in dict_reag :
        list_reg_agg_2 = dict_reag[reg_import]
        s1=pd.DataFrame(np.zeros_like(new_dcba.loc['FR','FR']),index=new_dcba.loc['FR','FR'].index, columns = new_dcba.loc['FR','FR'].columns)
        #print(s1.columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                #print(reg1,reg2)
                s1 += dcba.loc[reg1,reg2]
        
        for line in s1.index :
            for col in s1.columns :
                new_dcba.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
        
new_dcba

region                   FR                                       \
sector          Agriculture Crude coal     Crude oil Natural gas   
region stressor                                                    
FR     CO2         4.796711   0.013821  3.149139e-05    2.894532   
       CH4        15.816831   0.000037  3.567591e-04    0.275431   
       N2O         9.715682   0.000093  5.603963e-07    0.007834   
       SF6         0.028011   0.000003  2.607044e-08    0.000477   
       HFC         1.487352   0.000147  1.475096e-06    0.024675   
...                     ...        ...           ...         ...   
EU     CH4         3.091531   0.000355  3.570174e-05    0.028653   
       N2O         1.669396   0.000009  3.575776e-07    0.003974   
       SF6         0.036347   0.000070  1.357071e-07    0.004541   
       HFC         0.165743   0.000017  5.493366e-07    0.007612   
       PFC         0.037457   0.000003  1.171452e-07    0.003592   

region                                                                         \
sector          Extractive industry Biomass_industry  Clothing Heavy_industry   
region stressor                                                                 
FR     CO2                 0.005818         9.866832  0.394570      13.261457   
       CH4                 0.000362         9.923741  0.075845       0.632317   
       N2O                 0.000127         6.010542  0.027877       0.196020   
       SF6                 0.000480         0.036355  0.000561       0.024654   
       HFC                 0.025029         1.894707  0.028297       1.641677   
...                             ...              ...       ...            ...   
EU     CH4                 0.000253         3.277704  0.257131       0.483021   
       N2O                 0.000066         1.750927  0.063645       0.107515   
       SF6                 0.000240         0.161043  0.018176       0.144685   
       HFC                 0.002472         0.397501  0.067395       0.288126   
       PFC                 0.000667         0.115331  0.035363       0.228889   

region                                   ...             EU               \
sector          Construction Automobile  ... Heavy_industry Construction   
region stressor                          ...                               
FR     CO2         14.029825   2.479608  ...       2.171746     1.680882   
       CH4          2.537475   0.319671  ...       0.190533     0.301424   
       N2O          0.968326   0.082451  ...       0.080943     0.142388   
       SF6          0.054173   0.003670  ...       0.004446     0.003608   
       HFC          2.874809   0.197846  ...       0.279117     0.206061   
...                      ...        ...  ...            ...          ...   
EU     CH4          0.871780   0.738130  ...       6.795545    22.206020   
       N2O          0.255239   0.142840  ...       1.141034     4.279811   
       SF6          0.133053   0.134929  ...       3.033871     3.721052   
       HFC          0.337786   0.252778  ...       5.002919     8.423468   
       PFC          0.224067   0.148901  ...       3.246368     3.064161   

region                                                                    \
sector          Automobile Oth transport equipment Machinery Electronics   
region stressor                                                            
FR     CO2        1.452006                0.551266  1.903225    0.424498   
       CH4        0.182520                0.077270  0.192376    0.046600   
       N2O        0.053332                0.018148  0.064001    0.016325   
       SF6        0.002249                0.000771  0.003134    0.000585   
       HFC        0.124646                0.038295  0.173902    0.030852   
...                    ...                     ...       ...         ...   
EU     CH4        2.374384                0.658824  4.628122    1.109084   
       N2O        0.421663                0.136212  0.813450    0.213190   
       SF6 

In [14]:
np.sum(new_dcba['FR'])

sector
Agriculture                 56.440243
Crude coal                   0.038900
Crude oil                    0.003644
Natural gas                  4.840542
Extractive industry          0.051217
Biomass_industry            60.594626
Clothing                    19.969169
Heavy_industry              43.953967
Construction                52.219384
Automobile                  26.930765
Oth transport equipment      4.506169
Machinery                   18.373178
Electronics                 12.142085
Fossil fuels                31.492568
Electricity and heat        20.088773
Transport services          56.315201
Composite                  171.788343
dtype: float64

## MCVE

In [15]:
list_reg_mcve = ['FR', 'United States', 'United Kingdom']
list_reg_mcve_new = ['FR','Other']
sectors_list_mcve = ['Agriculture','Composite']
dict_mcve = {'FR':['FR'],'Other' : ['United States', 'United Kingdom']}

ghg_list_mcve = ['CO2', 'CH4']

multi_reg = []
multi_sec = []
for reg in list_reg_mcve :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve),
                                   len(sectors_list_mcve)*len(list_reg_mcve))),
                              index =new_index,columns = new_col)

multi_reg = []
multi_sec = []
for reg in list_reg_mcve_new :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve_new :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve_new = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve_new),
                                   len(sectors_list_mcve)*len(list_reg_mcve_new))),
                              index =new_index,columns = new_col)
from random import randint
for col in dcba_mcve.columns:
    dcba_mcve[col]=dcba_mcve.apply(lambda x: randint(0,5), axis=1)


print(dcba_mcve)

for reg_export in dict_mcve :
    list_reg_agg_1 = dict_mcve[reg_export]
    for reg_import in dict_mcve :
        list_reg_agg_2 = dict_mcve[reg_import]
        s1=pd.DataFrame(np.zeros_like(dcba_mcve_new.loc['FR','FR']),index=dcba_mcve_new.loc['FR','FR'].index, columns = dcba_mcve_new.loc['FR','FR'].columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                #print(reg1,reg2)
                s1 += dcba_mcve.loc[reg1,reg2].copy()
        for line in s1.index :
            for col in s1.columns :
                dcba_mcve_new.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
dcba_mcve_new


region                           FR           United States            \
sector                  Agriculture Composite   Agriculture Composite   
region         stressor                                                 
FR             CO2                3         2             1         0   
               CH4                3         5             2         5   
United States  CO2                2         5             5         3   
               CH4                3         2             2         4   
United Kingdom CO2                5         1             5         1   
               CH4                5         2             3         2   

region                  United Kingdom            
sector                     Agriculture Composite  
region         stressor                           
FR             CO2                   1         4  
               CH4                   0         2  
United States  CO2                   1         5  
               CH4                  

region                   FR                 Other          
sector          Agriculture Composite Agriculture Composite
region stressor                                            
FR     CO2              3.0       2.0         2.0       4.0
       CH4              3.0       5.0         2.0       7.0
Other  CO2              7.0       6.0        15.0      12.0
       CH4              8.0       4.0        12.0       7.0

## CVE

In [16]:
list_reg_mcve = reg_list #['FR', 'United States', 'United Kingdom']
list_reg_mcve_new = list_reg_reag_new #['FR','Other']
sectors_list_mcve = sectors_list #['Agriculture','Composite']
dict_mcve = dict_reag #{'FR':['FR'],'Other' : ['United States', 'United Kingdom']}

ghg_list_mcve = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']

multi_reg = []
multi_sec = []
for reg in list_reg_mcve :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

dcba_mcve = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve),
                                   len(sectors_list_mcve)*len(list_reg_mcve))),
                              index =new_index,columns = new_col)

multi_reg = []
multi_sec = []
for reg in list_reg_mcve_new :
    for sec in sectors_list_mcve :
        multi_reg.append(reg)
        multi_sec.append(sec)
arrays = [multi_reg, multi_sec]
new_col = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

multi_reg2 = []
multi_ghg = []
for reg in list_reg_mcve_new :
    for ghg in ghg_list_mcve :
        multi_reg2.append(reg)
        multi_ghg.append(ghg)
arrays2 = [multi_reg2, multi_ghg]
new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

#dcba_mcve_new = pd.DataFrame(np.zeros((len(ghg_list_mcve)*len(list_reg_mcve_new),
#                                   len(sectors_list_mcve)*len(list_reg_mcve_new))),
#                              index =new_index,columns = new_col)
dcba_mcve_new = pd.DataFrame(None, index =new_index,columns = new_col)
dcba_mcve_new.fillna(value=0,inplace=True)


from random import randint
for col in dcba_mcve.columns:
    dcba_mcve[col]=dcba_mcve.apply(lambda x: randint(0,5), axis=1)


for reg_export in dict_mcve :
    list_reg_agg_1 = dict_mcve[reg_export]
    for reg_import in dict_mcve :
        list_reg_agg_2 = dict_mcve[reg_import]
        s1=pd.DataFrame(np.zeros_like(dcba_mcve_new.loc['FR','FR']),index=dcba_mcve_new.loc['FR','FR'].index, columns = dcba_mcve_new.loc['FR','FR'].columns)
        for reg1 in list_reg_agg_1 :
            for reg2 in list_reg_agg_2 :
                s1 += dcba_mcve.loc[reg1,reg2]
        for line in s1.index :
            for col in s1.columns :
                dcba_mcve_new.at[(reg_export,line),(reg_import,col)]=s1.loc[line,col]
dcba_mcve_new

region                   FR                                   \
sector          Agriculture Crude coal Crude oil Natural gas   
region stressor                                                
FR     CO2                1          3         5           2   
       CH4                4          0         4           0   
       N2O                3          0         5           2   
       SF6                4          0         5           5   
       HFC                0          3         3           3   
...                     ...        ...       ...         ...   
EU     CH4               68         67        57          60   
       N2O               50         64        79          61   
       SF6               67         55        63          78   
       HFC               54         76        63          61   
       PFC               57         58        74          51   

region                                                                        \
sector          Extractive industry Biomass_industry Clothing Heavy_industry   
region stressor                                                                
FR     CO2                        1                4        5              5   
       CH4                        1                3        4              4   
       N2O                        2                0        4              4   
       SF6                        1                1        0              5   
       HFC                        5                2        5              2   
...                             ...              ...      ...            ...   
EU     CH4                       60               63       51             71   
       N2O                       58               50       66             66   
       SF6                       64               55       64             54   
       HFC                       58               58       57             60   
       PFC                       62               67       76             84   

region                                   ...             EU               \
sector          Construction Automobile  ... Heavy_industry Construction   
region stressor                          ...                               
FR     CO2                 5          1  ...             54           65   
       CH4                 0          3  ...             50           61   
       N2O                 3          3  ...             61           65   
       SF6                 0          5  ...             76           71   
       HFC                 4          1  ...             58           55   
...                      ...        ...  ...            ...          ...   
EU     CH4                62         49  ...           1685         1665   
       N2O                54         72  ...           1666         1703   
       SF6                74         54  ...           1626         1569   
       HFC                49         54  ...           1750         1672   
       PFC                70         78  ...           1649         1678   

region                                                                    \
sector          Automobile Oth transport equipment Machinery Electronics   
region stressor                                                            
FR     CO2              50                      72        64          62   
       CH4              60                      61        65          84   
       N2O              62                      65        64          67   
       SF6              40                      73        77          68   
       HFC              68                      70        67          66   
...                    ...                     ...       ...         ...   
EU     CH4            1733                    1727      1768        1734   
       N2O            1750                    1714      1701        1646   
       SF6            1682                    1670      1715        1754   
     

# Contenu de "reference" à remplacer :

In [110]:
def get_attribute(obj, path_string):
    parts = path_string.split('.')
    final_attribute_index = len(parts)-1
    current_attribute = obj
    i = 0
    for part in parts:
        new_attr = getattr(current_attribute, part, None)
        if current_attribute is None:
            print('Error %s not found in %s' % (part, current_attribute))
            return None
        if i == final_attribute_index:
            return getattr(current_attribute, part)
        current_attribute = new_attr
        i += 1
        
def set_attribute(obj, path_string, new_value):
    parts = path_string.split('.')
    final_attribute_index = len(parts)-1
    current_attribute = obj
    i = 0
    for part in parts:
        new_attr = getattr(current_attribute, part, None)
        if current_attribute is None:
            print('Error %s not found in %s' % (part, current_attribute))
            break
        if i == final_attribute_index:
            setattr(current_attribute, part, new_value)
        current_attribute = new_attr
        i+=1

def compute_new_multi_index(ind_names,sectors_list,ghg_list,conso_sect_list,list_reg_reag_new):

    if ind_names == ('region', 'conso') :
        multi_reg = []
        multi_sec = []
        for reg in list_reg_reag_new :
            for sec in conso_sect_list :
                multi_reg.append(reg)
                multi_sec.append(sec)
        arrays = [multi_reg, multi_sec]
        new_index = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

    elif ind_names == ('region', 'sector') :
        multi_reg = []
        multi_sec = []
        for reg in list_reg_reag_new :
            for sec in sectors_list :
                multi_reg.append(reg)
                multi_sec.append(sec)
        arrays = [multi_reg, multi_sec]
        new_index = pd.MultiIndex.from_arrays(arrays, names=('region', 'sector'))

    elif ind_names == ('region', 'stressor') :
        multi_reg = []
        multi_ghg = []
        for reg in list_reg_reag_new :
            for ghg in ghg_list :
                multi_reg.append(reg)
                multi_ghg.append(ghg)
        arrays2 = [multi_reg, multi_ghg]
        new_index = pd.MultiIndex.from_arrays(arrays2, names=('region', 'stressor'))

    elif ind_names == ('stressor',) :
        new_index = pd.Index(ghg_list,name='stressor')

    elif ind_names == ('indout',) :
        new_index = pd.Index(['indout'],name='indout')

    return new_index


def replace_reagg_scenar_attributes(scenario,reaggregation_matrix):

    ghg_list = ['CO2', 'CH4', 'N2O', 'SF6', 'HFC', 'PFC']

    conso_sect_list = ['Final consumption expenditure by households',
                    'Final consumption expenditure by non-profit organisations serving households (NPISH)',
                    'Final consumption expenditure by government',
                    'Gross fixed capital formation',
                    'Changes in inventories',
                    'Changes in valuables',
                    'Exports: Total (fob)']

    list_reg_reag_new=list(reaggregation_matrix.columns[2:])

    #create dic for region reaggregation :
    dict_reag={}
    dict_reag['FR']=['FR']
    for reg_agg in list(reaggregation_matrix.columns[3:]):
        list_reg_agg = []
        for i in reaggregation_matrix.index:
            reg = reaggregation_matrix.iloc[i].loc['Country name']
            if reaggregation_matrix[reg_agg].iloc[i] == 1:
                list_reg_agg.append(reg)
        dict_reag[reg_agg]=list_reg_agg
    dict_reag

    to_replace_list = ['A','L','x','Y','Z','ghg_emissions.D_cba','ghg_emissions.D_pba','ghg_emissions.D_exp',
                       'ghg_emissions.D_imp','ghg_emissions.F','ghg_emissions.F_Y','ghg_emissions.M','ghg_emissions.S',
                       'ghg_emissions.S_Y','ghg_emissions_desag.D_cba','ghg_emissions_desag.D_pba','ghg_emissions_desag.D_exp',
                       'ghg_emissions_desag.D_imp','ghg_emissions_desag.F','ghg_emissions_desag.F_Y','ghg_emissions_desag.M',
                       'ghg_emissions_desag.S','ghg_emissions_desag.S_Y']

    dict_index_reag = {'A':[('region','sector'),('region','sector')],
                'L':[('region','sector'),('region','sector')],
                'x':[('region','sector'),('indout',)],
                'Y':[('region','sector'),('region','conso')],
                'Z':[('region','sector'),('region','sector')],
                'ghg_emissions.D_cba':[('stressor',),('region','sector')],
                'ghg_emissions.D_pba':[('stressor',),('region','sector')],
                'ghg_emissions.D_exp':[('stressor',),('region','sector')],
                'ghg_emissions.D_imp':[('stressor',),('region','sector')],
                'ghg_emissions.F':[('stressor',),('region','sector')],
                'ghg_emissions.F_Y':[('stressor',),('region','conso')],
                'ghg_emissions.M':[('stressor',),('region','sector')],
                'ghg_emissions.S':[('stressor',),('region','sector')],
                'ghg_emissions.S_Y':[('stressor',),('region','conso')],
                'ghg_emissions_desag.D_cba':[('region','stressor'),('region','sector')],
                'ghg_emissions_desag.D_pba':[('region','stressor'),('region','sector')],
                'ghg_emissions_desag.D_exp':[('region','stressor'),('region','sector')],
                'ghg_emissions_desag.D_imp':[('region','stressor'),('region','sector')],
                'ghg_emissions_desag.F':[('stressor',),('region','sector')],
                'ghg_emissions_desag.F_Y':[('stressor',),('region','conso')],
                'ghg_emissions_desag.M':[('stressor',),('region','sector')],
                'ghg_emissions_desag.S':[('stressor',),('region','sector')],
                'ghg_emissions_desag.S_Y':[('stressor',),('region','conso')]}

    dict_func_reag = {'A':'sum','L':'sum',#to be checked
                'x':'sum','Y':'sum','Z':'sum',
                'ghg_emissions.D_cba':'sum',
                'ghg_emissions.D_pba':'sum',
                'ghg_emissions.D_exp':'sum',
                'ghg_emissions.D_imp':'sum',
                'ghg_emissions.F':'sum', #to be checked
                'ghg_emissions.F_Y':'sum',
                'ghg_emissions.M':'mean',
                'ghg_emissions.S':'mean',
                'ghg_emissions.S_Y':'mean',
                'ghg_emissions_desag.D_cba':'sum',
                'ghg_emissions_desag.D_pba':'sum',
                'ghg_emissions_desag.D_exp':'sum',
                'ghg_emissions_desag.D_imp':'sum',
                'ghg_emissions_desag.F':'sum', #to be checked
                'ghg_emissions_desag.F_Y':'sum',
                'ghg_emissions_desag.M':'mean',
                'ghg_emissions_desag.S':'mean',
                'ghg_emissions_desag.S_Y':'mean'}

    for attr in to_replace_list:
        #print(attr)
        mat = get_attribute(scenario,attr)
        
        new_ind = compute_new_multi_index(dict_index_reag[attr][0],list(scenario.get_sectors()),ghg_list,conso_sect_list,list_reg_reag_new)
        new_col = compute_new_multi_index(dict_index_reag[attr][1],list(scenario.get_sectors()),ghg_list,conso_sect_list,list_reg_reag_new)

        new_mat = pd.DataFrame(None,index = new_ind, columns = new_col)
        new_mat.fillna(value=0.,inplace=True)
        
        dict_reshape={('region','sector'):(11,17),
             ('indout',):(1,),
             ('region','conso'):(11,7),
             ('region','stressor'):(11,6),
             ('stressor',):(6,)}

        for line in np.reshape(new_ind,dict_reshape[dict_index_reag[attr][0]]) :
            if np.shape(line)==() or np.shape(line)==(1,) :
                elt_line = line
            else :
                elt_line = line[0][0]
            if 'region' in new_ind.names :
                list_reg_agg_1 = dict_reag[elt_line]

                for col in np.reshape(new_col,dict_reshape[dict_index_reag[attr][1]]) :
                    if np.shape(col)==() or np.shape(col)==(1,) :
                        elt_col = col
                    else :
                        elt_col = col[0][0]

                    if 'region' in new_col.names :
                        list_reg_agg_2 = dict_reag[elt_col]
                        s1=pd.DataFrame(np.zeros_like(new_mat.loc[elt_line,elt_col]),
                                        index=new_mat.loc[elt_line,elt_col].index, 
                                        columns = new_mat.loc[elt_line,elt_col].columns, 
                                        dtype=np.float64)
                        count=0
                        for reg1 in list_reg_agg_1 :
                            for reg2 in list_reg_agg_2 :
                                s1 += mat.loc[reg1,reg2]
                                count+=1
                        if dict_func_reag[attr] == 'mean':
                            s1 = s1/count

                        for line_s1 in s1.index :
                            for col_s1 in s1.columns :
                                new_mat.at[(elt_line,line_s1),(elt_col,col_s1)]=s1.loc[line_s1,col_s1]

                    else :
                        s1=pd.DataFrame(np.zeros_like(new_mat.loc[elt_line]),
                                        index=new_mat.loc[elt_line].index, 
                                        columns = new_mat.loc[elt_line].columns,
                                        dtype=np.float64)
                        count=0
                        for reg1 in list_reg_agg_1 :
                                s1 += mat.loc[reg1]
                                count+=1
                        
                        if dict_func_reag[attr] == 'mean':
                            s1 = s1/count

                        for line_s1 in s1.index :
                            for col_s1 in s1.columns :
                                new_mat.at[(elt_line,line_s1),(elt_col,col_s1)]=s1.loc[line_s1,col_s1]
                        
                        

            elif 'region' in new_col.names:
                for col in np.reshape(new_col,dict_reshape[dict_index_reag[attr][1]]) :

                    if np.shape(col)==() or np.shape(col)==(1,) :
                        elt_col = col
                    else :
                        elt_col = col[0][0]
                    list_reg_agg_2 = dict_reag[elt_col]
                    s1=pd.DataFrame(np.zeros_like(new_mat.loc[:,elt_col]),
                                    index=new_mat.loc[:,elt_col].index,
                                    columns = new_mat.loc[:,elt_col].columns,
                                    dtype=np.float64)
                    count=0
                    for reg2 in list_reg_agg_2 :
                        s1 += mat.loc[:,reg2]
                        count+=1

                    if dict_func_reag[attr] == 'mean':
                        s1 = s1/count

                    for line_s1 in s1.index :
                        for col_s1 in s1.columns :
                            new_mat.at[(elt_line,line_s1),(elt_col,col_s1)]=s1.loc[line_s1,col_s1]

            else :
                for col in np.reshape(new_col,dict_reshape[dict_index_reag[attr][1]]) :
                    
                    elt_col = col[0][0]
                    s1=pd.DataFrame(mat.loc[elt_line,elt_col],index=new_mat.loc[elt_line,elt_col].index,
                                    columns = new_mat.loc[elt_line,elt_col].columns, dtype=np.float64)
                for line_s1 in s1.index :
                    for col_s1 in s1.columns :
                        new_mat.at[(elt_line,line_s1),(elt_col,col_s1)]=s1.loc[line_s1,col_s1]

                            

        set_attribute(scenario,attr,new_mat)
        #print(np.shape(new_mat))
    return

In [116]:
reference.get_regions()

Index(['FR', 'United Kingdom', 'United States', 'Asia and Row Europe',
       'Chinafrica', 'Turkey and RoW America', 'Pacific and RoW Middle East',
       'Brazil, Mexico and South Africa', 'Switzerland and Norway',
       'RoW Asia and Pacific', 'EU'],
      dtype='object', name='region')

In [105]:
reference = pymrio.parse_exiobase3(data_dir / ('reference' + '_' + concat_settings))
reference.calc_all()


# update extension calculations
reference.ghg_emissions_desag = Tools.recal_extensions_per_region(
	reference,
	'ghg_emissions'
)
ref2=reference.copy()

replace_reagg_scenar_attributes(reference,reaggregation_matrix = pd.read_excel(data_dir / 'agg_matrix_opti.xlsx', sheet_name = 'region_ref'))

A
(187, 187)
L
(187, 187)
x
(187, 1)
Y
(187, 77)
Z
(187, 187)
ghg_emissions.D_cba
(6, 187)
ghg_emissions.D_pba
(6, 187)
ghg_emissions.D_exp
(6, 187)
ghg_emissions.D_imp
(6, 187)
ghg_emissions.F
(6, 187)
ghg_emissions.F_Y
(6, 77)
ghg_emissions.M
(6, 187)
ghg_emissions.S
(6, 187)
ghg_emissions.S_Y
(6, 77)
ghg_emissions_desag.D_cba
(66, 187)
ghg_emissions_desag.D_pba
(66, 187)
ghg_emissions_desag.D_exp
(66, 187)
ghg_emissions_desag.D_imp
(66, 187)
ghg_emissions_desag.F
(6, 187)
ghg_emissions_desag.F_Y
(6, 77)
ghg_emissions_desag.M
(6, 187)
ghg_emissions_desag.S
(6, 187)
ghg_emissions_desag.S_Y
(6, 77)


In [106]:
for attr in to_replace_list :
    mat_ref = get_attribute(reference,attr)
    mat_ref2 = get_attribute(ref2,attr)
    print(attr)
    print('reference', np.shape(mat_ref), mat_ref.sum().sum())
    print('ref2', np.shape(mat_ref2), mat_ref2.sum().sum(),'\n')
        
#ref2.ghg_emissions_desag.D_pba#.sum().sum()

A
reference (187, 187) 468.39131929337304
ref2 (833, 833) 468.39131929337304 

L
reference (187, 187) 1805.3529479202168
ref2 (833, 833) 1805.352947920217 

x
reference (187, 1) 135059649.52913487
ref2 (833, 1) 135059649.52913487 

Y
reference (187, 77) 67585167.95048761
ref2 (833, 343) 67585167.95048764 

Z
reference (187, 187) 67474481.57866973
ref2 (833, 833) 67474481.57866971 

ghg_emissions.D_cba
reference (6, 187) 41453.763655427254
ref2 (6, 833) 41453.763655427254 

ghg_emissions.D_pba
reference (6, 187) 41453.76365542725
ref2 (6, 833) 41453.76365542726 

ghg_emissions.D_exp
reference (6, 187) 11619.285763208265
ref2 (6, 833) 11619.285763208265 

ghg_emissions.D_imp
reference (6, 187) 11619.285763208267
ref2 (6, 833) 11619.285763208265 

ghg_emissions.F
reference (6, 187) 41459.56455464967
ref2 (6, 833) 41459.56455464967 

ghg_emissions.F_Y
reference (6, 77) 5524.603140998271
ref2 (6, 343) 5524.603140998271 

ghg_emissions.M
reference (6, 187) 882.7169813343089
ref2 (6, 833) 229

In [123]:
sectors_list=list(reference.get_sectors())
reg_list=list(reference.get_regions())
demcat_list = list(reference.get_Y_categories())

In [126]:
counterfactual = reference.copy()
counterfactual.remove_extension('ghg_emissions_desag')
sectors,moves = scenar_bestv2()
#sectors,moves = scenar_pref_europev3()
for sector in sectors:
	counterfactual.Z,counterfactual.Y = Tools.shockv2(sectors,demcat_list,reg_list,counterfactual.Z,counterfactual.Y,moves[sector],sector)

counterfactual.A = None
counterfactual.x = None
counterfactual.L = None

# calculate counterfactual(s) system
counterfactual.calc_all()
#print(counterfactual.Z)
counterfactual.ghg_emissions_desag = Tools.recal_extensions_per_region(
	counterfactual,
	'ghg_emissions'
)

In [154]:
counterfactual.ghg_emissions_desag.D_cba.loc[:,'FR'].sum()#.sum()
counterfactual.x.sum()#.sum()
counterfactual.L.sum().sum()
reference.A.sum().sum()
counterfactual.Z.sum().sum()
counterfactual.Y.sum().sum()
reference.ghg_emissions.D_cba.sum().sum()

41453.763655427254

In [121]:
def get_least(sector,reloc):
	#par défaut on ne se laisse pas la possibilité de relocaliser en FR
	M = reference.ghg_emissions_desag.M.sum()
	#compute the part of the french economy that depends on broad activities, for each sector :
	final_demfr= reference.Y['FR'].drop(['FR']).sum(axis=1)
	interdemfr=reference.Z['FR'].drop(['FR']).sum(axis=1)
	import_demand_FR = (final_demfr+interdemfr).sum(level=1)

	regs = list(reference.get_regions())[1:]

	if reloc:
		regs = list(reference.get_regions())
	ind=0
	for i in range(1,len(regs)):
		if M[regs[i],sector] < M[regs[ind],sector] and reference.Z.loc[regs[i]].drop(columns=regs[i]).sum(axis=1).loc[sector] > import_demand_FR.loc[sector] : # pour choisir une région comme région de report, elle doit au moins déjà exporter l'équivalent de la partie importée de la demande française
			ind=i
	return regs[ind]

def sort_by_content(sector,regs):
	#sort all regions by carbon content of a sector
	#carbon contents
	M = reference.ghg_emissions_desag.M.sum()
	carbon_content_sector = [M[regs[i],sector] for i in range(len(regs))]
	index_sorted = np.argsort(carbon_content_sector)
	return index_sorted

def worst_moves(sector,reloc):
	if reloc:
		regs = list(reference.get_regions())
	else:
		regs = list(reference.get_regions())[1:] #remove FR
	index_sorted = list(reversed(sort_by_content(sector,regs)))
	sectors_list = list(reference.get_sectors())
	demcats = list(reference.get_Y_categories())

	#compute the part of the french economy that depends on broad activities, for this sector :
	final_demfr= reference.Y['FR'].drop(['FR']).sum(axis=1).sum(level=1).loc[sector]
	interdemfr=reference.Z['FR'].drop(['FR']).sum(axis=1).sum(level=1).loc[sector]
	import_demand_FR = final_demfr+interdemfr
	#part de chaque secteur français dans les importations intermédiaires françaises depuis un secteur étranger
	part_prod_secteurs =[] 
	part_dem_secteurs = []
	for sec in sectors_list:
		part_prod_secteurs.append(reference.Z[('FR',sec)].drop(['FR']).sum(level=1).loc[sector]/import_demand_FR)
	for dem in demcats:
		part_dem_secteurs.append(reference.Y[('FR',dem)].drop(['FR']).sum(level=1).loc[sector]/import_demand_FR)
	
	#parts des importations françaises *totales pour un secteur* à importer depuis le 1er best, 2nd best...
	nbreg=len(regs)
	nbsect=len(sectors_list)
	nbdemcats = len(demcats)
	parts_sects = np.zeros((nbreg,nbsect))
	parts_demcats = np.zeros((nbreg,nbdemcats))
	#construction of french needs of imports
	totalfromsector = np.zeros(nbsect)
	totalfinalfromsector = np.zeros(nbdemcats)
	for j in range(nbsect):
		#sum on regions of imports of imports of sector for french sector j
		totalfromsector[j] = np.sum([reference.Z['FR'].drop('FR')[sectors_list[j]].loc[(regs[k],sector)] for k in range(nbreg)]) 
	for j in range(nbdemcats):
		totalfinalfromsector[j] = np.sum([reference.Y['FR'].drop('FR')[demcats[j]].loc[(regs[k],sector)] for k in range(nbreg)])

	remaining_reg_export = np.zeros(nbreg)
	for i in range(nbreg):
		my_best = regs[index_sorted[i]] #region with ith lowest carbon content for this sector
		reg_export = reference.Z.drop(columns=my_best).sum(axis=1).loc[(my_best,sector)] #exports from this reg/sec
		remaining_reg_export[index_sorted[i]] = reg_export
		for j in range(nbsect):
			if np.sum(parts_sects[:,j]) < totalfromsector[j] and remaining_reg_export[index_sorted[i]] >0:
				#if imp demand from sector j is not satisfied and if my_best can still export some sector
				alloc=0
				if remaining_reg_export[index_sorted[i]]>totalfromsector[j]:
					alloc=totalfromsector[j]
				else:
					alloc=remaining_reg_export[index_sorted[i]]
				parts_sects[index_sorted[i],j] = alloc
				remaining_reg_export[index_sorted[i]] -= alloc
				
		for j in range(nbdemcats):
			#idem for final demand categories
			if remaining_reg_export[index_sorted[i]] >0 and np.sum(parts_demcats[:,j]) < totalfinalfromsector[j]:
				alloc=0
				if remaining_reg_export[index_sorted[i]] > totalfinalfromsector[j]:
					alloc = totalfinalfromsector[j]
				else:
					alloc = remaining_reg_export[index_sorted[i]]
				parts_demcats[index_sorted[i],j] = alloc
				remaining_reg_export[index_sorted[i]] -= alloc
	return parts_sects,parts_demcats,index_sorted

def best_moves(sector,reloc):
	if reloc:
		regs = list(reference.get_regions())
	else:
		regs = list(reference.get_regions())[1:] #remove FR
	index_sorted = sort_by_content(sector,regs)
	sectors_list = list(reference.get_sectors())
	demcats = list(reference.get_Y_categories())

	#compute the part of the french economy that depends on broad activities, for this sector :
	final_demfr= reference.Y['FR'].drop(['FR']).sum(axis=1).sum(level=1).loc[sector]
	interdemfr=reference.Z['FR'].drop(['FR']).sum(axis=1).sum(level=1).loc[sector]
	import_demand_FR = final_demfr+interdemfr
	#part de chaque secteur français dans les importations intermédiaires françaises depuis un secteur étranger
	part_prod_secteurs =[] 
	part_dem_secteurs = []
	for sec in sectors_list:
		part_prod_secteurs.append(reference.Z[('FR',sec)].drop(['FR']).sum(level=1).loc[sector]/import_demand_FR)
	for dem in demcats:
		part_dem_secteurs.append(reference.Y[('FR',dem)].drop(['FR']).sum(level=1).loc[sector]/import_demand_FR)
	
	#parts des importations françaises *totales pour un secteur* à importer depuis le 1er best, 2nd best...
	nbreg=len(regs)
	nbsect=len(sectors_list)
	nbdemcats = len(demcats)
	parts_sects = np.zeros((nbreg,nbsect))
	parts_demcats = np.zeros((nbreg,nbdemcats))
	#construction of french needs of imports
	totalfromsector = np.zeros(nbsect)
	totalfinalfromsector = np.zeros(nbdemcats)
	for j in range(nbsect):
		#sum on regions of imports of imports of sector for french sector j
		totalfromsector[j] = np.sum([reference.Z['FR'].drop('FR')[sectors_list[j]].loc[(regs[k],sector)] for k in range(nbreg)]) 
	for j in range(nbdemcats):
		totalfinalfromsector[j] = np.sum([reference.Y['FR'].drop('FR')[demcats[j]].loc[(regs[k],sector)] for k in range(nbreg)])

	#export capacities of each regions
	remaining_reg_export = np.zeros(nbreg)
	for i in range(nbreg):
		my_best = regs[index_sorted[i]] #region with ith lowest carbon content for this sector
		reg_export = reference.Z.drop(columns=my_best).sum(axis=1).loc[(my_best,sector)] #exports from this reg/sec
		remaining_reg_export[index_sorted[i]] = reg_export

	for j in range(nbsect):
		covered = 0
		for i in range(nbreg):
			if covered < totalfromsector[j] and remaining_reg_export[index_sorted[i]] >0:
				#if imp demand from sector j is not satisfied and if my_best can still export some sector
				if remaining_reg_export[index_sorted[i]]>totalfromsector[j]-covered:
					alloc=totalfromsector[j]-covered
				else:
					alloc=remaining_reg_export[index_sorted[i]]
				parts_sects[index_sorted[i],j] = alloc
				remaining_reg_export[index_sorted[i]] -= alloc
				covered+=alloc
				
	for j in range(nbdemcats):
		#idem for final demand categories
		covered = 0
		for i in range(nbreg):
			if  covered < totalfinalfromsector[j] and remaining_reg_export[index_sorted[i]] >0 :
				if remaining_reg_export[index_sorted[i]] > totalfinalfromsector[j]-covered:
					alloc = totalfinalfromsector[j]-covered
				else:
					alloc = remaining_reg_export[index_sorted[i]]
				parts_demcats[index_sorted[i],j] = alloc
				remaining_reg_export[index_sorted[i]] -= alloc
				covered+=alloc
	return parts_sects,parts_demcats,index_sorted

def scenar_bestv2(reloc=False):
	sectors_list = list(reference.get_sectors())
	moves = {}
	for sector in sectors_list:
		part_sec, part_dem,index_sorted = best_moves(sector,reloc)
		moves[sector] = {'parts_sec' : part_sec, 'parts_dem':part_dem, 'sort':index_sorted, 'reloc':reloc}
	return sectors_list, moves

def scenar_worstv2(reloc=False):
	sectors_list = list(reference.get_sectors())
	moves = {}
	for sector in sectors_list:
		part_sec, part_dem,index_sorted = worst_moves(sector,reloc)
		moves[sector] = {'parts_sec' : part_sec, 'parts_dem':part_dem, 'sort':index_sorted, 'reloc':reloc}
	return sectors_list, moves


def get_worst(sector,reloc):
	#par défaut on ne se laisse pas la possibilité de relocaliser en FR
	M = reference.ghg_emissions_desag.M.sum()
	regs = list(reference.get_regions())[1:]
	if reloc:
		regs = list(reference.get_regions())
	ind=0
	for i in range(1,len(regs)):
		if M[regs[i],sector] > M[regs[ind],sector]:
			ind=i
	return regs[ind]

#construction du scénario least intense
def scenar_best(reloc=False,deloc=False):
	sectors_list = list(reference.get_sectors())
	sectors_gl = []
	moves_gl = []
	for sector in sectors_list:
		best = get_least(sector,reloc)
		if deloc:
			for i in range(len(list(reference.get_regions()))-1):
				sectors_gl.append(sector)
		else:
			for i in range(len(list(reference.get_regions()))-2):
				sectors_gl.append(sector)
		for reg in list(reference.get_regions()):
			if deloc:
				if reg!=best:
					moves_gl.append([reg,best])
			else:
				if reg!=best :
					if reg!='FR':
						moves_gl.append([reg,best])
	quantities = [1 for i in range(len(sectors_gl))]
	return sectors_gl, moves_gl, quantities

def scenar_worst(reloc=False,deloc=False):
	sectors_list = list(reference.get_sectors())
	sectors_gl = []
	moves_gl = []
	for sector in sectors_list:
		worst = get_worst(sector,reloc)
		if deloc:
			for i in range(len(list(reference.get_regions()))-1):
				sectors_gl.append(sector)
		else:
			for i in range(len(list(reference.get_regions()))-2):
				sectors_gl.append(sector)
		for reg in list(reference.get_regions()):
			if deloc:
				if reg!=worst:
					moves_gl.append([reg,worst])
			else:
				if reg!=worst :
					if reg!='FR':
						moves_gl.append([reg,worst])
	quantities = [1 for i in range(len(sectors_gl))]
	return sectors_gl, moves_gl, quantities


def scenar_pref_europe():
	nbreg = len(list(reference.get_regions()))
	sectors = (nbreg-2)*list(reference.get_sectors())
	quantities = [1 for i in range(len(sectors)) ]
	moves =[]
	for i in range(nbreg):
		reg = reference.get_regions()[i]
		if reg != 'Europe' and reg != 'FR':
			for j in range(len(list(reference.get_sectors()))):
				moves.append([reg,'Europe'])
	return sectors,moves,quantities

def scenar_pref_europev3(reloc=False):
	if reloc:
		regs = list(reference.get_regions())
	else:
		regs = list(reference.get_regions())[1:] #remove FR
	sectors_list = list(reference.get_sectors())
	demcats = list(reference.get_Y_categories())
	nbdemcats=len(demcats)
	nbreg=len(regs)
	moves = {}
	for i in range(nbsect):
		#initialization of outputs
		parts_sects = {}
		parts_dem = {}
		for r in regs:
			parts_sects[r] = np.zeros(nbsect)
			parts_dem[r] = np.zeros(nbdemcats)

		#construction of french needs of imports
		totalfromsector = np.zeros(nbsect)
		totalfinalfromsector = np.zeros(nbdemcats)
		for j in range(nbsect):
			#sum on regions of imports of imports of sector for french sector j
			totalfromsector[j] = np.sum([reference.Z['FR'].drop('FR')[sectors_list[j]].loc[(regs[k],sectors_list[i])] for k in range(nbreg)]) 
		for j in range(nbdemcats):
			totalfinalfromsector[j] = np.sum([reference.Y['FR'].drop('FR')[demcats[j]].loc[(regs[k],sectors_list[i])] for k in range(nbreg)])
		
		# exports capacity of all regions for sector i
		reg_export = {}
		for r in range(nbreg):
			reg_export[regs[r]] = reference.Z.drop(columns=regs[r]).sum(axis=1).loc[(regs[r],sectors_list[i])] #exports from this reg/sec
		
		remaining_reg_export_UE = reg_export['Europe']
		for j in range(nbsect):
			if totalfromsector[j] !=0:
				if remaining_reg_export_UE > 0:
					#if europe can still export some sector[i]
					if remaining_reg_export_UE>totalfromsector[j]:
						alloc=totalfromsector[j]
					else:
						alloc= reference.Z.loc[('Europe',sectors_list[i]),('FR',sectors_list[j])] #tout ou rien ici
					parts_sects['Europe'][j] = alloc
					remaining_reg_export_UE -= alloc
					#remove from other regions a part of what has been assigned to the EU
					# this part corresponds to the part of the country in original french imports for sector j 
					for r in regs:
						if r != 'Europe':
							parts_sects[r][j] =  reference.Z.loc[(r,sectors_list[i]),('FR',sectors_list[j])]* (1- alloc /totalfromsector[j])

		for j in range(nbdemcats):
			if totalfinalfromsector[j] != 0:
				if remaining_reg_export_UE > 0:
					#if europe can still export some sector[i]
					if remaining_reg_export_UE>totalfinalfromsector[j]:
						alloc=totalfinalfromsector[j]
					else:
						alloc=reference.Y.loc[('Europe',sectors_list[i]),('FR',demcats[j])] #tout ou rien ici
					parts_dem['Europe'][j] = alloc
					remaining_reg_export_UE -= alloc
					#remove from other regions a part of what has been assigned to the EU
					# this part corresponds to the part of the country in original french imports for sector j 
					for r in regs:
						if r != 'Europe':
							parts_sects[r][j] =  reference.Y.loc[(r,sectors_list[i]),('FR',demcats[j])]* (1- alloc /totalfinalfromsector[j])

		moves[sectors_list[i]] = {'parts_sec' : parts_sects, 'parts_dem':parts_dem, 'sort':[i for i in range(len(regs))], 'reloc':reloc}
	return sectors_list,moves
